# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [5]:
# extract Orders (pd.read_csv also works with .zip files)
orders = pd.read_csv('Orders.zip')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [4]:
# SUB PROBLEM 1: aggreggate the amount_spent for unique customers

# Step 1: First, I am going to create a new, smaller df that only contains the relevant columns for my analysis,
# including for each invoice: customerID, Country, amount_spent

orders_smaller = orders[['CustomerID', 'Country', 'amount_spent']]
orders_smaller.head()

,CustomerID,Country,amount_spent
0,17850,United Kingdom,15.30
1,17850,United Kingdom,20.34
2,17850,United Kingdom,22.00
3,17850,United Kingdom,20.34
4,17850,United Kingdom,20.34


In [8]:
# Step 2: Now I am going to aggregate the amount spent per customer

amount_customer = orders_smaller.groupby(['CustomerID']).agg({'amount_spent':sum})
amount_customer.head()

,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40


In [14]:
# SUB PROBLEM 2: select customers whose aggregated amount_spent is in a given quantile range

# Step 1: First, I store in a variable the min. amount that customers must have spent to be considered
# in the 95th and the 75th percentile

percentile_95 = amount_customer.amount_spent.quantile(0.95)
percentile_75 = amount_customer.amount_spent.quantile(0.75)


In [ ]:
# Note: I think I may have got the calculation of the quantiles wrong (I will review it).
# If so, that may impact the results of number of customers per level in the following problems.
# However, regardless of the figures, I think the steps and code for the following problems should be ok

In [17]:
# Now we create a function that will return:
## - VIP for customers whose amount_spent is in the 95th percentile
## - Preferred for customers in the 75th-95th
## - Regular for all other customers

def customer_level(x):
    if x >= percentile_95:
        return 'VIP'
    elif x >= percentile_75:
        return 'Preferred'
    else:
        return 'Regular'

customer_level(3000) # This is just to test it with different values

'Preferred'

In [23]:
# Now we can apply the function to the column amount_spent
# And then, create a new column with the result of the function (see below)

amount_customer['amount_spent'].apply(customer_level)

CustomerID
12346          VIP
12347    Preferred
12348    Preferred
12349    Preferred
12350      Regular
           ...    
18280      Regular
18281      Regular
18282      Regular
18283    Preferred
18287    Preferred
Name: amount_spent, Length: 4339, dtype: object

In [22]:
# Lastly, we add the new column with the labels
# Now we have all customers labeled as VIP and Preferred (or else, Regular) as requested
# We can use these labels to then filter and extract the customers of a given level only

amount_customer['customer_level'] = amount_customer['amount_spent'].apply(customer_level)
amount_customer

,amount_spent,customer_level
CustomerID,,
12346,77183.60,VIP
12347,4310.00,Preferred
12348,1797.24,Preferred
12349,1757.55,Preferred
12350,334.40,Regular
...,...,...
18280,180.60,Regular
18281,80.82,Regular
18282,178.05,Regular


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [25]:
# First, I am joining the countries to the table above.
# There are many ways to do it, I'll do it with merge

customer_country = orders_smaller[['CustomerID', 'Country']]

levels_countries = pd.merge(left=amount_customer, 
                                right=customer_country,
                                how='inner', 
                                left_on = 'CustomerID',
                                right_on = 'CustomerID')

levels_countries



,CustomerID,amount_spent,customer_level,Country
0,12346,77183.60,VIP,United Kingdom
1,12347,4310.00,Preferred,Iceland
2,12347,4310.00,Preferred,Iceland
3,12347,4310.00,Preferred,Iceland
4,12347,4310.00,Preferred,Iceland
...,...,...,...,...
397919,18287,1837.28,Preferred,United Kingdom
397920,18287,1837.28,Preferred,United Kingdom
397921,18287,1837.28,Preferred,United Kingdom
397922,18287,1837.28,Preferred,United Kingdom


In [39]:
# This gives the number of VIP customers per country:
VIP_countries = (levels_countries.customer_level == 'VIP').groupby(levels_countries.Country).sum()

#And this will give us the country which is at the top:
VIP_countries.nlargest(1)

# The country with more VIP customers is United Kingdom

Country
United Kingdom    84185
Name: customer_level, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [42]:
# Similar to the code in Q2, but the conditional must be transformed, so it meets the condition when 
# customer level is VIP OR customer level is Preferred

VIP_Preferred_countries = ((levels_countries.customer_level == 'VIP') | (levels_countries.customer_level == 'Preferred')).groupby(levels_countries.Country).sum()
VIP_Preferred_countries.nlargest(1)

# The country with more VIP and Preferred customers combined is the UK

Country
United Kingdom    221635
Name: customer_level, dtype: int64